### Recursively Rename Files After Validation

### **Test before use on real data**

In [1]:
import os
import pandas as pd
from pathlib import Path
import glob  # To find the `_Validation.txt` files dynamically

In [ ]:
# Paths
prev_val_clips = "/mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/uncertainty/it_1_s0.5_t0.1/"
new_clips_dir = "/mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/uncertainty/it_1_s1_t0.01/"

In [ ]:
# Function to extract core filename (ignoring score)
def extract_core_filename(filename):
    parts = Path(filename).stem.split('_')  # Remove extension and split filename
    return '_'.join(parts[1:6])  # Extract offset, siteID, survey night, date, time

# Iterate through each class folder in the previous validation clips
for class_folder in os.listdir(prev_val_clips):
    old_class_path = os.path.join(prev_val_clips, class_folder)
    new_class_path = os.path.join(new_clips_dir, class_folder)

    # Ensure the class exists in both directories
    if not os.path.isdir(old_class_path) or not os.path.isdir(new_class_path):
        continue

    # Find the validation .txt file (assuming one per class)
    old_txt_files = glob.glob(os.path.join(old_class_path, "*_Validation.txt"))
    new_txt_files = glob.glob(os.path.join(new_class_path, "*_Validation.txt"))

    if len(old_txt_files) == 0:
        print(f"⚠ No previous validation file found in {class_folder}, skipping...")
        continue
    if len(new_txt_files) == 0:
        print(f"⚠ No new validation file found in {class_folder}, skipping...")
        continue

    old_txt_file = old_txt_files[0]  # Pick the old validation file
    new_txt_file = new_txt_files[0]  # Pick the new validation file

    # Load old validation data
    df_old = pd.read_csv(old_txt_file, delimiter='\t', usecols=["Begin File", "Eval", "Annotation"])

    # Create a dictionary mapping core filenames to their Eval & Annotation
    validated_clips = {
        extract_core_filename(row["Begin File"]): (row["Eval"], row["Annotation"])
        for _, row in df_old.iterrows()
    }

    # Load new validation data
    df_new = pd.read_csv(new_txt_file, delimiter='\t')

    # Ensure required columns exist
    if "Begin File" not in df_new.columns:
        print(f"⚠ Skipping {class_folder} due to missing 'Begin File' column in new validation file.")
        continue

    # Add Eval and Annotation columns if not present
    if "Eval" not in df_new.columns:
        df_new["Eval"] = ""
    if "Annotation" not in df_new.columns:
        df_new["Annotation"] = ""

    # Update new validation file with matched Eval & Annotation values
    for index, row in df_new.iterrows():
        core_name = extract_core_filename(row["Begin File"])
        if core_name in validated_clips:
            df_new.at[index, "Eval"] = validated_clips[core_name][0]  # Copy Eval
            df_new.at[index, "Annotation"] = validated_clips[core_name][1]  # Copy Annotation

    # Save the updated validation file
    df_new.to_csv(new_txt_file, sep='\t', index=False)

    print(f"✔ Updated {new_txt_file} with preserved validation data.")

print("✅ All annotation files updated successfully.")
